In [ ]:
import sys
import pandas as pd
import numpy as np
from config import config
from experiment import get_trimmed_data, plot_period_cumulative_wealth, plot_full_cumulative_wealth

sys.path.append('.')

window_size = config.get('parameter.window_size')
cutoff = config.get('parameter.cutoff')
start_date = config.get('data.period.start')
end_date = config.get('data.period.end')
start_year = start_date[:4]
end_year = end_date[:4]

# Display configs
print(f'Dataset: {config.get('data.dataset')}')
print(f'Period: {start_date} to {end_date}')

print(f'Window size: {window_size}')
print(f'Cutoff: {cutoff}')

data_daily = get_trimmed_data()

# The rate of return matrix, fill the missing values with 100%
Return = data_daily.values.T.astype(float)
Return[Return < -99]
Gross_return = (Return + 100) / 100 # Gross Return
print(Gross_return.shape)

window_size = config.get('parameter.window_size')

# Check the first few rows of the dataset
data_daily.head()


Dataset: FF48
Period: 198501 to 202412
Window size: 120
Cutoff: 5


TypeError: unsupported operand type(s) for &: 'str' and 'str'

In [ ]:
if config.get('algorithm.ucb'):
    from algorithms.ucb import UCB
    ucb = UCB(Gross_return, window_size=window_size, cutoff=cutoff)
    ucb.run()

In [ ]:
if config.get('algorithm.psr'):
    from algorithms.psr import PSR
    psr = PSR(Gross_return, window_size=window_size, cutoff=cutoff)
    psr.run()

In [ ]:
if config.get('algorithm.psr-ucb'):
    from algorithms.psr_ucb import PSRUCB
    psr_ucb = PSRUCB(Gross_return, window_size=window_size, cutoff=cutoff)
    psr_ucb.run()

In [ ]:
if config.get('algorithm.pw-ucb'):
    from algorithms.pw_ucb import PWUCB
    pw_ucb = PWUCB(Gross_return, window_size=window_size, cutoff=cutoff)
    pw_ucb.run()

In [ ]:
# Baseline
constant_weight_rebalance = np.cumprod(Gross_return[:, window_size:].mean(axis=0))
equal_weight_portfolio = np.mean(np.cumprod(Gross_return[:, window_size:], axis=1), axis=0)

date = data_daily.index.values[window_size:]

# Minimum variance portfolio (MVP)


In [ ]:
ew_reward = np.mean(Gross_return[:, window_size:], axis=0)

In [ ]:
# Save the results
all_returns = {"EW": ew_reward}

# add the results of each algorithm to the all_returns dictionary
if config.get('algorithm.ucb'):
    all_returns.update({"UCB": ucb.reward})
if config.get('algorithm.psr'):
    all_returns.update({"PSR": psr.reward})
if config.get('algorithm.psr-ucb'):
    all_returns.update({"PSR-UCB": psr_ucb.reward})
if config.get('algorithm.pw-ucb'):
    all_returns.update({"PW-UCB": pw_ucb.reward})

z = pd.DataFrame(all_returns) - 1
z = z.to_csv('./output/new_reward.csv')


In [ ]:
# Cumulative wealth
cumulative_wealths = {
    "CWR": constant_weight_rebalance,
    "EW": equal_weight_portfolio,
}
if config.get('algorithm.ucb'):
    cumulative_wealths.update({"UCB": ucb.get_cumulative_wealth()})
if config.get('algorithm.psr'): 
    cumulative_wealths.update({"PSR": psr.get_cumulative_wealth()})
if config.get('algorithm.psr-ucb'):
    cumulative_wealths.update({"PSR-UCB": psr_ucb.get_cumulative_wealth()})
if config.get('algorithm.pw-ucb'):
    cumulative_wealths.update({"PW-UCB": pw_ucb.get_cumulative_wealth()})

results = pd.DataFrame(cumulative_wealths)
# Cumulative wealth
results_pct = results.pct_change().dropna()


In [ ]:
results_pct

In [ ]:
# Annualized standard deviation
results_pct.std() * 100 * np.sqrt(252)

In [ ]:
results.index = date

In [ ]:
results_pct.index = date[1:]

In [ ]:
years = pd.date_range(start=start_year, end=end_year, freq='YE').year
years = [str(year) for year in years]

In [ ]:
yearly_sharpes = {}

for col in results_pct.columns:
    col_ = {}
    for a in range(1, len(years)):
        col_[years[a]] = np.sqrt(252) * np.mean(results_pct.loc[years[a-1]:years[a], col]) / np.std(results_pct.loc[years[a-1]:years[a], col])
    yearly_sharpes[col] = col_

In [ ]:
yearly_sharpes = pd.DataFrame(yearly_sharpes)

In [ ]:
results_pct.to_csv('./output/pct_changes.csv')

In [ ]:
# Annualized Mean Sharpe Ratio
yearly_sharpes.mean().sort_values(ascending=False)

In [ ]:
# Yearly Annualized Mean Sharpe Ratio
((np.mean(results_pct)/np.std(results_pct))*np.sqrt(252)).sort_values(ascending=False)

In [ ]:
# Cumulative wealths
results.loc["2015":"2024", :]

In [ ]:
results.to_csv('./output/results.csv')

In [ ]:
# Plotting full period cumulative wealth
plot_full_cumulative_wealth(results=results)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

DASH_STYLES = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2)]

# Plotting periodic cumulative wealth
# plot_period_cumulative_wealth(results_pct=results_pct)
num_algorithms = len(results_pct.columns)

start_year_int = int(start_year)
end_year_int = int(end_year)

# divide the years into 10-year intervals    
interval = config.get('plot.interval')
num_plots = (end_year_int - start_year_int + 1) / interval
# get integer, 3.0 -> 3, 3.5 -> 4
num_plots = int(num_plots) if num_plots.is_integer() else int(num_plots) + 1

wealths = {}
for i in range(num_plots):
    begin = str(start_year_int + i * interval)
    stop = str(min(start_year_int + (i+1) * interval - 1, end_year_int))
    print(f"Begin: {begin}, Stop: {stop}")
    print(results_pct.loc[begin:stop, :].index[0])
    wealth = np.cumprod(pd.DataFrame(np.concatenate(
        (np.ones(num_algorithms).reshape(-1, num_algorithms), 
            results_pct.loc[begin:stop, :].values+1), axis=0), 
            columns=results_pct.columns, 
            index=pd.date_range(start=results_pct.loc[begin:stop, :].index[0]-pd.Timedelta("1d"), 
                            end=results_pct.loc[begin:stop, :].index[-1], 
                            periods=results_pct.loc[begin:stop, :].shape[0]+1).date))
    
    wealths.update({f"{begin}-{stop}": wealth})
    

sns.set_theme(style='whitegrid', font_scale=3.3)

for period, wealth_period in wealths.items():
    plt.figure(figsize=(60, 25))
    ax = sns.lineplot(data=wealth_period, palette="bright", linewidth=2.7, dashes=DASH_STYLES)
    ax.set(xlabel='Date', ylabel='Cumulative Wealth', title=f"{config.get('data.dataset')} Algorithm Comparison ({period})")
    ax.figure.savefig(f"{config.get('plot.output_dir')}/cw_{period}.png")
    plt.close()





In [ ]:
# Graphs
# num_algorithms = len(results.columns)

# #wealth_75_84 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(num_algorithms).reshape(-1, array_size), results_pct.loc["1975":"1984", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["1975":"1984", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["1975":"1984", :].index[-1], periods=results_pct.loc["1975":"1984", :].shape[0]+1).date))
# wealth_85_94 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(num_algorithms).reshape(-1, num_algorithms), results_pct.loc["1985":"1994", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["1985":"1994", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["1985":"1994", :].index[-1], periods=results_pct.loc["1985":"1994", :].shape[0]+1).date))
# wealth_95_04 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(num_algorithms).reshape(-1, num_algorithms), results_pct.loc["1995":"2004", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["1995":"2004", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["1995":"2004", :].index[-1], periods=results_pct.loc["1995":"2004", :].shape[0]+1).date))
# wealth_05_14 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(num_algorithms).reshape(-1, num_algorithms), results_pct.loc["2005":"2014", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["2005":"2014", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["2005":"2014", :].index[-1], periods=results_pct.loc["2005":"2014", :].shape[0]+1).date))
# wealth_15_24 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(num_algorithms).reshape(-1, num_algorithms), results_pct.loc["2015":"2024", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["2015":"2024", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["2015":"2024", :].index[-1], periods=results_pct.loc["2015":"2024", :].shape[0]+1).date))

# import seaborn as sns
# import matplotlib.pyplot as plt

# dash_styles = ["",
#                (4, 1.5),
#                (1, 1),
#                (3, 1, 1.5, 1),
#                (5, 1, 1, 1),
#                (5, 1, 2, 1, 2, 1),
#                (2, 2, 3, 1.5),
#                (1, 2.5, 3, 1.2)]

# sns.set_theme(style="whitegrid", font_scale=3.2)
# plt.figure(figsize=(60, 25))
# ax = sns.lineplot(data=results, palette="bright", linewidth=2.7, dashes=dash_styles)
# ax.set(xlabel="Date", ylabel='Cumulative Wealth', title="FF48 Algorithm Comparison");
# ax.figure.savefig("./output/cw_overall.png")

# wealth_periods = {
#     # "1975-1984": wealth_75_84,
#     # "1985-1994": wealth_85_94,
#     # "1995-2004": wealth_95_04,
#     "2005-2014": wealth_05_14,
#     "2015-2024": wealth_15_24
# }

# sns.set_theme(style="whitegrid", font_scale=3.3)


# for period_name, wealth_period in wealth_periods.items():
#     plt.figure(figsize=(60, 25))
#     ax = sns.lineplot(data=wealth_period, palette="bright", linewidth=2.7, dashes=dash_styles)
#     ax.set(xlabel="Date", ylabel='Cumulative Wealth', title=f"FF48 Algorithm Comparison ({period_name})")
#     ax.figure.savefig(f'./output/cw_{period_name}.png')
#     plt.close()


yearly_sharpes.to_csv('./output/year_sharpes.csv')
